In [ ]:
import mediapipe as mp
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict # To store angles per frame
import joblib
from scipy.signal import find_peaks
from matplotlib.colors import ListedColormap

import os
# List all files
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

%matplotlib inline

> NOTE: Currently only works with videos looking to the right


# Values for Video and Recommendations


In [ ]:
# VIDEO_PATH = "/kaggle/input/cycling-stock-video/istockphoto-1313936692-640_adpp_is.mp4"
VIDEO_PATH = "Hands_Top.mov"  # NEEDS TO BE DOWNLOADED
# VIDEO_PATH = "/kaggle/input/cycling-dennis/dennis_cycling.mp4"
# VIDEO_PATH = "/kaggle/input/cycling-dennis/dennis_cycling_full.mov"
# VIDEO_PATH = "/kaggle/input/cycling-sebastian/Hands_Top.mov"

VIDEO_DURATION = 10  # seconds

In [ ]:
# Define optimal ranges (these are general guidelines and can be adjusted)
# Angle Hip-Knee-Ankle, 180 is straight
OPTIMAL_KNEE_EXTENSION_BOTTOM = (140, 150)
# Angle Hip-Knee-Ankle, 180 is straight (less common to define)
OPTIMAL_KNEE_EXTENSION_TOP = (70, 80)
# Angle Hip-Shoulder-Horizontal (40 aggressive, 50 endurance)
OPTIMAL_TORSO_ANGLE_TO_HORIZONTAL = (40, 50)
# Angle Shoulder-Elbow-Wrist (180 is straight)
OPTIMAL_ELBOW_ANGLE = (150, 170)
# Angle Hip-Shoulder-Elbow
OPTIMAL_SHOULDER_ANGLE = (85, 90)

# Default Imports and Objects


In [ ]:
# Helper Classes
class Point:
    """Represents a 2D point with x and y coordinates."""

    def __init__(self, x: float, y: float):
        self.x = x
        self.y = y

    def __str__(self):
        # Round to 2 decimal places
        return f"({self.x:.2f}, {self.y:.2f})"

    def __sub__(self, other):
        """Subtracts another Point from this Point, returning a new Point (vector)."""
        return Point(self.x - other.x, self.y - other.y)

    def __add__(self, other):
        """Adds two Points, returning a new Point (vector)."""
        return Point(self.x + other.x, self.y + other.y)

    def get_from_gpu(self):
        """Attempts to move tensor coordinates from GPU to CPU if they are on GPU."""
        if hasattr(self.x, 'cpu'):
            self.x = self.x.cpu().item()  # .item() to get scalar value
        if hasattr(self.y, 'cpu'):
            self.y = self.y.cpu().item()  # .item() to get scalar value


class FrameLandmarks:
    """Stores landmarks for a single frame."""

    def __init__(self):
        self.nose: Point = None
        self.right_wrist: Point = None
        self.right_elbow: Point = None
        self.right_shoulder: Point = None
        self.right_hip: Point = None
        self.right_knee: Point = None
        self.right_ankle: Point = None
        self.right_foot_index: Point = None
        self.right_heel: Point = None
        # Add left side for potential future use (e.g., bilateral analysis)
        self.left_wrist: Point = None
        self.left_elbow: Point = None
        self.left_shoulder: Point = None
        self.left_hip: Point = None
        self.left_knee: Point = None
        self.left_ankle: Point = None
        self.left_foot_index: Point = None
        self.left_heel: Point = None

    def from_gpu_to_cpu(self):
        """Converts all Point objects within this frame's landmarks from GPU to CPU."""
        for attr_name in dir(self):
            if isinstance(getattr(self, attr_name), Point):
                getattr(self, attr_name).get_from_gpu()


class AllLandmarks:
    """Stores a list of FrameLandmarks for all processed frames."""

    def __init__(self):
        self.frames_landmarks: list[FrameLandmarks] = []

    def append_frame(self, frame_landmarks: FrameLandmarks):
        self.frames_landmarks.append(frame_landmarks)

    def get_landmark_series(self, landmark_name: str) -> list[Point]:
        """Returns a list of Points for a specific landmark across all frames."""
        series = []
        for frame_lm in self.frames_landmarks:
            if hasattr(frame_lm, landmark_name) and getattr(frame_lm, landmark_name) is not None:
                series.append(getattr(frame_lm, landmark_name))
        return series

    def clear(self):
        """Clears all stored landmarks."""
        self.frames_landmarks = []


def plot_image_with_points(image, frame_landmarks: FrameLandmarks, with_foot_details=False):
    """Plots an image with detected body landmarks."""
    plt.figure(figsize=(16, 10))  # to set the figure size
    plt.imshow(image)

    marker_size = 5  # Increased marker size for better visibility

    # List of tuples: (landmark_point_attribute, color, marker_style)
    # Define connections for visual lines
    connections = [
        ("right_shoulder", "right_elbow"),
        ("right_elbow", "right_wrist"),
        ("right_shoulder", "right_hip"),
        ("right_hip", "right_knee"),
        ("right_knee", "right_ankle"),
    ]

    points_to_plot = [
        (frame_landmarks.nose, 'r', 'o', "Nose"),
        (frame_landmarks.right_wrist, 'r', 'o', "Right Wrist"),
        (frame_landmarks.right_elbow, 'g', 'o', "Right Elbow"),
        (frame_landmarks.right_shoulder, 'b', 'o', "Right Shoulder"),
        (frame_landmarks.right_hip, 'c', 'o', "Right Hip"),
        (frame_landmarks.right_knee, 'm', 'o', "Right Knee"),
        (frame_landmarks.right_ankle, 'y', 'o', "Right Ankle"),
    ]

    for p, color, marker, label in points_to_plot:
        if p:
            plt.plot(p.x, p.y, color=color, linestyle="None",
                     marker=marker, markersize=marker_size, label=label)

    # Draw connections
    for p1_attr, p2_attr in connections:
        p1 = getattr(frame_landmarks, p1_attr, None)
        p2 = getattr(frame_landmarks, p2_attr, None)
        if p1 and p2:
            plt.plot([p1.x, p2.x], [p1.y, p2.y],
                     color='lightgray', linewidth=1)

    if with_foot_details:
        if frame_landmarks.right_foot_index:
            plt.plot(frame_landmarks.right_foot_index.x, frame_landmarks.right_foot_index.y, 'darkblue',
                     linestyle="None", marker="o", markersize=marker_size, label="Right Foot Index")
        if frame_landmarks.right_heel:
            plt.plot(frame_landmarks.right_heel.x, frame_landmarks.right_heel.y, 'darkgreen',
                     linestyle="None", marker="o", markersize=marker_size, label="Right Heel")
        if frame_landmarks.right_foot_index and frame_landmarks.right_heel:
            plt.plot([frame_landmarks.right_foot_index.x, frame_landmarks.right_heel.x],
                     [frame_landmarks.right_foot_index.y,
                         frame_landmarks.right_heel.y],
                     color='lightgray', linewidth=1)

    plt.legend()
    plt.grid(True)
    plt.title("Keypoints Detection")
    plt.show()


all_frames_landmarks = AllLandmarks()

In [ ]:
!wget -O cycling_video.webm https://www.shutterstock.com/shutterstock/videos/1075982534/preview/stock-footage-side-view-of-asian-woman-cyclist-is-exercising-in-the-house-by-cycling-on-trainer-and-play-online.webm 

In [ ]:
# Check if the Video exists
if not os.path.exists(VIDEO_PATH):
    raise TypeError("Video not found.")

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise TypeError("Could not open video file.")

VIDEO_LENGTH = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
VIDEO_FPS = int(cap.get(cv2.CAP_PROP_FPS))
VIDEO_WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
VIDEO_HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()  # Release the cap after getting info

print(f"Video length: {VIDEO_LENGTH} frames")
print(f"Video FPS: {VIDEO_FPS}")
print(f"Video width: {VIDEO_WIDTH}")
print(f"Video height: {VIDEO_HEIGHT}")

In [ ]:
# Helper functions for the angles

def calculate_angle(a: Point, b: Point, c: Point) -> float:
    """Calculates the angle (in degrees) between three points, with b as the vertex."""
    # Create vectors from the vertex point b
    vec1 = a - b
    vec2 = c - b

    # Calculate dot product
    dot_product = vec1.x * vec2.x + vec1.y * vec2.y

    # Calculate magnitudes
    mag1 = np.sqrt(vec1.x**2 + vec1.y**2)
    mag2 = np.sqrt(vec2.x**2 + vec2.y**2)

    if mag1 == 0 or mag2 == 0:
        return 0.0  # Avoid division by zero

    cosine_angle = dot_product / (mag1 * mag2)
    # Ensure cosine_angle is within [-1, 1] to prevent arccos errors due to floating point inaccuracies
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    angle_rad = np.arccos(cosine_angle)
    angle_deg = np.degrees(angle_rad)
    return angle_deg


def print_angle_stats(angles: list[float], angle_name: str):
    """Prints statistics for a list of angles."""
    if not angles:
        print(f"No {angle_name} angles calculated.")
        return
    print(f"\n--- {angle_name} Angle Statistics ---")
    print(f"Min angle: {min(angles):.2f}°")
    print(f"Max angle: {max(angles):.2f}°")
    print(f"Mean angle: {np.mean(angles):.2f}°")
    print(f"Median angle: {np.median(angles):.2f}°")
    print(f"Standard deviation: {np.std(angles):.2f}°")
    print("-" * (len(angle_name) + 22))


def get_image_at_index(index: int):
    """Retrieves a specific frame from the video."""
    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        print("Error opening video stream or file")
        raise TypeError

    if index >= VIDEO_LENGTH or index < 0:
        print(f'Invalid frame index: {index}. Video length: {VIDEO_LENGTH}')
        cap.release()
        return None

    cap.set(cv2.CAP_PROP_POS_FRAMES, index)
    ret, frame = cap.read()
    cap.release()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        return frame
    return None


def show_video_at_index(index: int, title: str):
    """Displays a specific frame from the video with a title."""
    frame = get_image_at_index(index)
    if frame is not None:
        plt.figure(figsize=(16, 10))  # to set the figure size
        plt.imshow(frame)
        plt.title(title)
        plt.axis('off')  # Hide axes for cleaner image
        plt.show()
    else:
        print(f"Could not retrieve frame at index {index}")


def add_text_to_image(image, text: str, pos: int, font_scale: float = 0.8, thickness: int = 1):
    """Adds text to an image at a specified position with adjustable font size and thickness."""
    height_offset = VIDEO_HEIGHT / 20  # Adjust offset for better spacing
    # Use cv2.LINE_AA for anti-aliased text
    cv2.putText(image, text,
                (10, int(height_offset * pos)),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale,  # Adjustable font scale
                (255, 0, 0),  # Red color
                thickness,  # Adjustable thickness
                cv2.LINE_AA)
    return image


def plot_angles_over_time(
    angles_data: dict[str, list[float]],
    title_prefix: str,
    separate_figures: bool = False,
    save: bool = True,
    optimal_ranges: dict[str, tuple[float, float]] | None = None,
    knee_peaks=None,
    max_knee_angles=None,
):
    """
    Plots angle values over time. Can plot all angles in subplots in one figure
    or each angle in a separate figure, with optional optimal range highlighting.
    Also adds peaks for "Knee Angle (Hip-Knee-Ankle)".

    Args:
        angles_data: A dictionary where keys are angle names (str) and values are lists of angle values (float).
        title_prefix: A string prefix for the plot titles (e.g., "MediaPipe" or "YOLOv8").
        separate_figures: If True, each angle will be plotted in its own subplot.
        save: If True, saves the figure(s) to a PNG file.
        optimal_ranges: Optional dict mapping angle names to (min, max) optimal range to highlight on plots.
    """
    if not angles_data:
        print(f"No angle data provided for {title_prefix} to plot.")
        return

    num_angles = len(angles_data)
    if num_angles == 0:
        print(f"No angle data provided for {title_prefix} to plot.")
        return

    if separate_figures:
        # Dynamically determine rows and columns for subplots
        rows = int(np.ceil(num_angles / 2)) if num_angles > 0 else 1
        cols = 2 if num_angles > 1 else 1

        fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
        # Ensure axes is always iterable, even for a single subplot
        axes = axes.flatten() if num_angles > 1 else [axes]

        for i, (angle_name, angles) in enumerate(angles_data.items()):
            if not angles:
                print(f"No data for {angle_name}. Skipping plot.")
                continue

            ax = axes[i]
            ax.plot(angles)
            ax.set_title(f"{angle_name}")
            ax.set_xlabel("Frame Number")
            ax.set_ylabel("Angle (°)")
            ax.grid(True)

            # Draw optimal range if available
            if optimal_ranges and angle_name in optimal_ranges:
                min_val, max_val = optimal_ranges[angle_name]
                ax.axhspan(min_val, max_val, color='green',
                           alpha=0.2, label='Optimal Range')
                ax.legend()

            # Add peaks specifically for "Knee Angle (Hip-Knee-Ankle)"
            if angle_name == "Knee Angle (Hip-Knee-Ankle)":
                if knee_peaks is not None and max_knee_angles is not None:
                    # knee_angles = np.array(angles)
                    # peaks, _ = find_peaks(knee_angles, distance=peak_detection_distance)
                    # Plot peaks as 'x' markers
                    ax.plot(knee_peaks, max_knee_angles, "x",
                            color='red', markersize=8, label='Peaks')
                    ax.legend()

        # Remove any unused subplots
        for j in range(i + 1, len(axes)):
            fig.delaxes(axes[j])

        fig.suptitle(f"{title_prefix} Angles Over Time", fontsize=16)
        # Adjust layout to prevent title overlap
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        if save:
            plt.savefig(f'{title_prefix}.png')
        plt.show()
    else:  # Plot all angles in one figure
        plt.figure(figsize=(15, 6))

        # Corrected way to get a discrete colormap:
        # Get the 'tab10' colormap object
        cmap = plt.colormaps.get_cmap('tab10')
        # Create a ListedColormap with the desired number of colors from the 'tab10' colormap
        # This ensures we have distinct colors for each line up to num_angles
        # Use the first num_angles colors
        colors = ListedColormap(cmap.colors[:num_angles])

        # Track if knee angle peaks have been added to avoid duplicate legends
        knee_peaks_added = False

        for i, (angle_name, angles) in enumerate(angles_data.items()):
            if angles:
                line_color = colors(i)
                # Plot the angle series
                plt.plot(angles, label=angle_name, color=line_color)

                # Add optimal range highlight
                if optimal_ranges and angle_name in optimal_ranges:
                    min_val, max_val = optimal_ranges[angle_name]
                    # Use the same color as the line for the optimal range highlight
                    plt.axhspan(min_val, max_val, color=line_color, alpha=0.2)
                else:
                    print(
                        f"No optimal range for {angle_name}. Skipping highlight.")

                # Add peaks for "Knee Angle (Hip-Knee-Ankle)"
                if angle_name == "Knee Angle (Hip-Knee-Ankle)":
                    if knee_peaks is not None and max_knee_angles is not None:
                        plt.plot(knee_peaks, max_knee_angles, "x", color='red', markersize=8,
                                 label='Knee Angle Peaks' if not knee_peaks_added else "_nolegend_")
                        knee_peaks_added = True  # Mark that peaks have been added
                    # knee_angles = np.array(angles)
                    # peaks, _ = find_peaks(knee_angles, distance=peak_detection_distance)
                    # # Plot peaks as 'x' markers. Add label only once.
                    # plt.plot(peaks, knee_angles[peaks], "x", color='red', markersize=8, label='Knee Angle Peaks' if not knee_peaks_added else "_nolegend_")
                    # knee_peaks_added = True # Mark that peaks have been added

        plt.title(title_prefix)
        plt.xlabel("Frame Number")
        plt.ylabel("Angle (degrees)")
        plt.grid(True)
        plt.legend()  # Display legend for all lines and the peak marker
        if save:
            plt.savefig(f'{title_prefix}.png')
        plt.show()

# Mediapipe Functions


In [ ]:
# Functions and Classes

# Define MediaPipe Landmark Indices (for clarity)
class MPLandmark:
    NOSE = mp.solutions.pose.PoseLandmark.NOSE
    RIGHT_WRIST = mp.solutions.pose.PoseLandmark.RIGHT_WRIST
    RIGHT_ELBOW = mp.solutions.pose.PoseLandmark.RIGHT_ELBOW
    RIGHT_SHOULDER = mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER
    RIGHT_HIP = mp.solutions.pose.PoseLandmark.RIGHT_HIP
    RIGHT_KNEE = mp.solutions.pose.PoseLandmark.RIGHT_KNEE
    RIGHT_ANKLE = mp.solutions.pose.PoseLandmark.RIGHT_ANKLE
    RIGHT_FOOT_INDEX = mp.solutions.pose.PoseLandmark.RIGHT_FOOT_INDEX
    RIGHT_HEEL = mp.solutions.pose.PoseLandmark.RIGHT_HEEL
    # Add left side for completeness
    LEFT_WRIST = mp.solutions.pose.PoseLandmark.LEFT_WRIST
    LEFT_ELBOW = mp.solutions.pose.PoseLandmark.LEFT_ELBOW
    LEFT_SHOULDER = mp.solutions.pose.PoseLandmark.LEFT_SHOULDER
    LEFT_HIP = mp.solutions.pose.PoseLandmark.LEFT_HIP
    LEFT_KNEE = mp.solutions.pose.PoseLandmark.LEFT_KNEE
    LEFT_ANKLE = mp.solutions.pose.PoseLandmark.LEFT_ANKLE
    LEFT_FOOT_INDEX = mp.solutions.pose.PoseLandmark.LEFT_FOOT_INDEX
    LEFT_HEEL = mp.solutions.pose.PoseLandmark.LEFT_HEEL


def extract_landmarks_mediapipe(results, image_width: int, image_height: int) -> FrameLandmarks | None:
    """Extracts relevant landmarks from MediaPipe results into a FrameLandmarks object."""
    if not results.pose_landmarks:
        return None

    frame_lm = FrameLandmarks()
    pose_landmarks = results.pose_landmarks.landmark

    # Helper to safely get landmark and convert to Point
    def get_point(landmark_enum):
        lm = pose_landmarks[landmark_enum.value]
        # Only include if visible (confidence > threshold, MediaPipe uses visibility for this)
        if lm.visibility > 0.7:  # You can adjust this threshold
            return Point(lm.x * image_width, lm.y * image_height)
        return None

    frame_lm.nose = get_point(MPLandmark.NOSE)
    frame_lm.right_wrist = get_point(MPLandmark.RIGHT_WRIST)
    frame_lm.right_elbow = get_point(MPLandmark.RIGHT_ELBOW)
    frame_lm.right_shoulder = get_point(MPLandmark.RIGHT_SHOULDER)
    frame_lm.right_hip = get_point(MPLandmark.RIGHT_HIP)
    frame_lm.right_knee = get_point(MPLandmark.RIGHT_KNEE)
    frame_lm.right_ankle = get_point(MPLandmark.RIGHT_ANKLE)
    frame_lm.right_foot_index = get_point(MPLandmark.RIGHT_FOOT_INDEX)
    frame_lm.right_heel = get_point(MPLandmark.RIGHT_HEEL)

    # Optional: Add left side if you plan to use it
    # frame_lm.left_wrist = get_point(MPLandmark.LEFT_WRIST)
    # frame_lm.left_elbow = get_point(MPLandmark.LEFT_ELBOW)
    # frame_lm.left_shoulder = get_point(MPLandmark.LEFT_SHOULDER)
    # frame_lm.left_hip = get_point(MPLandmark.LEFT_HIP)
    # frame_lm.left_knee = get_point(MPLandmark.LEFT_KNEE)
    # frame_lm.left_ankle = get_point(MPLandmark.LEFT_ANKLE)
    # frame_lm.left_foot_index = get_point(MPLandmark.LEFT_FOOT_INDEX)
    # frame_lm.left_heel = get_point(MPLandmark.LEFT_HEEL)

    return frame_lm


def get_bike_fit_recommendations(angles_data: dict[str, list[float]],
                                 knee_angle_peaks_avg: float) -> list[str]:
    """
    Generates bike fit recommendations based on calculated angles and standard ranges.
    """
    recommendations = []

    # Get average angles
    avg_knee_angle = np.mean(angles_data.get(
        "Knee Angle (Hip-Knee-Ankle)", [0]))
    avg_torso_angle = np.mean(angles_data.get(
        "Torso Angle (Shoulder-Hip-Horizontal)", [0]))
    avg_elbow_angle = np.mean(angles_data.get(
        "Elbow Angle (Shoulder-Elbow-Wrist)", [0]))
    # avg_knee_angle = avg_knee_angle if avg_knee_angle <= 90 else 180 - avg_knee_angle
    # avg_torso_angle = avg_torso_angle if avg_torso_angle <= 90 else 180 - avg_torso_angle

    recommendations.append("--- Bike Fit Recommendations ---")

    # 1. Saddle Height (based on Knee Extension at bottom of stroke)
    recommendations.append(
        f"Your Knee Angle (Hip-Knee-Ankle) in the Peaks: {knee_angle_peaks_avg:.2f}")
    if knee_angle_peaks_avg < OPTIMAL_KNEE_EXTENSION_BOTTOM[0]:
        recommendations.append(
            f"• Saddle might be TOO LOW. Consider RAISING your saddle height. (Optimal: {OPTIMAL_KNEE_EXTENSION_BOTTOM[0]}-{OPTIMAL_KNEE_EXTENSION_BOTTOM[1]}°)")
    elif knee_angle_peaks_avg > OPTIMAL_KNEE_EXTENSION_BOTTOM[1]:
        recommendations.append(
            f"• Saddle might be TOO HIGH. Consider LOWERING your saddle height. (Optimal: {OPTIMAL_KNEE_EXTENSION_BOTTOM[0]}-{OPTIMAL_KNEE_EXTENSION_BOTTOM[1]}°)")
    else:
        recommendations.append(
            f"• Knee extension (bottom): Within optimal range. ({OPTIMAL_KNEE_EXTENSION_BOTTOM[0]}-{OPTIMAL_KNEE_EXTENSION_BOTTOM[1]}°)")

    # 2. Saddle Fore/Aft (less direct from 2D angle, but can be inferred)
    # IS CALCULATED SEPERATE
    # recommendations.append("• **Saddle fore/aft position** requires further analysis (e.g., KOPS - Knee Over Pedal Spindle).")

    # 3. Handlebar Reach / Torso Angle
    recommendations.append(
        f"Your Torso Angle (Shoulder-Hip-Horizontal) AVG: {avg_torso_angle:.2f}")
    if avg_torso_angle < OPTIMAL_TORSO_ANGLE_TO_HORIZONTAL[0]:
        recommendations.append(
            f"• Torso angle is TOO AGGRESSIVE. Consider RAISING handlebars or getting a shorter stem. (Optimal: {OPTIMAL_TORSO_ANGLE_TO_HORIZONTAL[0]}-{OPTIMAL_TORSO_ANGLE_TO_HORIZONTAL[1]}°)")
    elif avg_torso_angle > OPTIMAL_TORSO_ANGLE_TO_HORIZONTAL[1]:
        recommendations.append(
            f"• Torso angle is TOO UPRIGHT. Consider LOWERING handlebars or getting a longer stem (if more aggressive stance desired). (Optimal: {OPTIMAL_TORSO_ANGLE_TO_HORIZONTAL[0]}-{OPTIMAL_TORSO_ANGLE_TO_HORIZONTAL[1]}°)")
    else:
        recommendations.append(
            f"• Torso angle: Within optimal range. ({OPTIMAL_TORSO_ANGLE_TO_HORIZONTAL[0]}-{OPTIMAL_TORSO_ANGLE_TO_HORIZONTAL[1]}°)")

    # 4. Handlebar Width / Elbow Angle / Arm Bend
    recommendations.append(
        f"Your Elbow Angle (Shoulder-Elbow-Wrist) AVG: {avg_elbow_angle:.2f}")
    if avg_elbow_angle > OPTIMAL_ELBOW_ANGLE[1]:
        recommendations.append(
            f"• Elbows appear TOO STRAIGHT. Try to maintain a slight bend for comfort and control. This could indicate a reach issue or stiff arms. (Optimal: {OPTIMAL_ELBOW_ANGLE[0]}-{OPTIMAL_ELBOW_ANGLE[1]}°)")
    elif avg_elbow_angle < OPTIMAL_ELBOW_ANGLE[0]:
        recommendations.append(
            f"• Elbows are TOO BENT. This might indicate reach is too short or an overly aggressive position, consider lengthening reach. (Optimal: {OPTIMAL_ELBOW_ANGLE[0]}-{OPTIMAL_ELBOW_ANGLE[1]}°)")
    else:
        recommendations.append(
            f"• Elbow angle: Within optimal range. ({OPTIMAL_ELBOW_ANGLE[0]}-{OPTIMAL_ELBOW_ANGLE[1]}°)")

    # Add general advice
    recommendations.append("\n--- General Advice ---")
    recommendations.append(
        "• These are general recommendations. Individual comfort and flexibility are key.")
    recommendations.append(
        "• Consider consulting a professional bike fitter for personalized adjustments.")
    recommendations.append(
        "• Make small adjustments and test them on the bike.")

    return recommendations


def draw_angle_on_image_precise(image, p1: Point, p_vertex: Point, p2: Point, angle_value: float, color=(0, 255, 255), line_thickness=2, arc_radius=50, font_scale=0.7, font_thickness=2):
    """
    Alternative implementation that draws the angle arc using points along the arc.
    This gives more precise control over the arc direction.
    """
    if not all([p1, p_vertex, p2]):
        return image

    # Draw segments
    cv2.line(image, (int(p1.x), int(p1.y)),
             (int(p_vertex.x), int(p_vertex.y)), color, line_thickness)
    cv2.line(image, (int(p2.x), int(p2.y)),
             (int(p_vertex.x), int(p_vertex.y)), color, line_thickness)

    # Calculate vectors from vertex to the other points
    vec1 = np.array([p1.x - p_vertex.x, p1.y - p_vertex.y])
    vec2 = np.array([p2.x - p_vertex.x, p2.y - p_vertex.y])

    # Get the angle between vectors using atan2
    angle1 = np.arctan2(vec1[1], vec1[0])
    angle2 = np.arctan2(vec2[1], vec2[0])

    # Ensure we draw the smaller arc
    angle_diff = angle2 - angle1
    if angle_diff > np.pi:
        angle_diff -= 2 * np.pi
    elif angle_diff < -np.pi:
        angle_diff += 2 * np.pi

    # Create points along the arc
    num_points = max(int(abs(angle_diff) * 180 / np.pi / 5),
                     3)  # At least 3 points
    angles = np.linspace(angle1, angle1 + angle_diff, num_points)

    arc_points = []
    for angle in angles:
        x = int(p_vertex.x + arc_radius * np.cos(angle))
        y = int(p_vertex.y + arc_radius * np.sin(angle))
        arc_points.append((x, y))

    # Draw the arc using polylines
    arc_points = np.array(arc_points, dtype=np.int32)
    cv2.polylines(image, [arc_points], False, color,
                  line_thickness, cv2.LINE_AA)

    # Calculate bisector for text placement
    mid_angle = angle1 + angle_diff / 2
    text_offset_x = int(p_vertex.x + (arc_radius + 15) * np.cos(mid_angle))
    text_offset_y = int(p_vertex.y + (arc_radius + 15) * np.sin(mid_angle))

    # Add text with background for better readability
    text = f'{angle_value:.1f}'
    text_size = cv2.getTextSize(
        text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)[0]

    # Draw text background rectangle
    cv2.rectangle(image,
                  (text_offset_x - 2, text_offset_y - text_size[1] - 2),
                  (text_offset_x + text_size[0] + 2, text_offset_y + 5),
                  (0, 0, 0), -1)  # Black background

    # Draw the text
    cv2.putText(image, text,
                (text_offset_x, text_offset_y),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale, color, font_thickness, cv2.LINE_AA)

    return image


def show_summarized_image(image, frame_landmarks: FrameLandmarks, angles_data: dict[str, list[float]],
                          bottom_idx: int, top_idx: int, model_name: str, angles_text=True):
    """
    Shows a summarized image with average angles, specific knee angles, and bike fit recommendations,
    and draws the angles on the image.
    """
    summary_image = image.copy()

    knee_angles_list = angles_data.get("Knee Angle (Hip-Knee-Ankle)", [])
    knee_angle_bottom = knee_angles_list[bottom_idx] if 0 <= bottom_idx < len(
        knee_angles_list) else 0
    knee_angle_top = knee_angles_list[top_idx] if 0 <= top_idx < len(
        knee_angles_list) else 0

    avg_shoulder_angle = np.mean(angles_data.get("Shoulder Angle (Hip-Shoulder-Elbow)", [
                                 0])) if angles_data.get("Shoulder Angle (Hip-Shoulder-Elbow)") else 0
    avg_elbow_angle = np.mean(angles_data.get("Elbow Angle (Shoulder-Elbow-Wrist)", [
                              0])) if angles_data.get("Elbow Angle (Shoulder-Elbow-Wrist)") else 0
    avg_torso_angle = np.mean(angles_data.get("Torso Angle (Shoulder-Hip-Horizontal)", [
                              0])) if angles_data.get("Torso Angle (Shoulder-Hip-Horizontal)") else 0
    avg_knee_angle = np.mean(angles_data.get(
        "Knee Angle (Hip-Knee-Ankle)", [0])) if angles_data.get("Knee Angle (Hip-Knee-Ankle)") else 0
    avg_ankle_angle = np.mean(angles_data.get("Ankle Angle (Knee-Ankle-Foot Index)", [
                              0])) if angles_data.get("Ankle Angle (Knee-Ankle-Foot Index)") else 0

    text_font_scale = 0.6
    text_thickness = 1
    text_color = (255, 255, 255)

    if angles_text:
        line_pos = 1
        add_text_to_image(
            summary_image, f'Knee Angle (Bottom): {knee_angle_bottom:.2f}°', line_pos, text_font_scale, text_thickness)
        line_pos += 1
        add_text_to_image(
            summary_image, f'Knee Angle (Top): {knee_angle_top:.2f}°', line_pos, text_font_scale, text_thickness)
        line_pos += 1
        add_text_to_image(
            summary_image, f'Knee Angle (Avg): {avg_knee_angle:.2f}°', line_pos, text_font_scale, text_thickness)
        line_pos += 1
        add_text_to_image(
            summary_image, f'Shoulder Angle (Avg): {avg_shoulder_angle:.2f}°', line_pos, text_font_scale, text_thickness)
        line_pos += 1
        add_text_to_image(
            summary_image, f'Elbow Angle (Avg): {avg_elbow_angle:.2f}°', line_pos, text_font_scale, text_thickness)
        line_pos += 1
        add_text_to_image(
            summary_image, f'Torso Angle (Avg): {avg_torso_angle:.2f}°', line_pos, text_font_scale, text_thickness)
        line_pos += 1
        add_text_to_image(
            summary_image, f'Ankle Angle (Avg): {avg_ankle_angle:.2f}°', line_pos, text_font_scale, text_thickness)
        line_pos += 1

    # --- Draw the angles on the image ---
    # Draw Knee Angle (Hip-Knee-Ankle)
    if frame_landmarks.right_hip and frame_landmarks.right_knee and frame_landmarks.right_ankle:
        knee_angle_val = calculate_angle(
            frame_landmarks.right_hip, frame_landmarks.right_knee, frame_landmarks.right_ankle)
        summary_image = draw_angle_on_image_precise(summary_image,
                                                    frame_landmarks.right_hip,
                                                    frame_landmarks.right_knee,
                                                    frame_landmarks.right_ankle,
                                                    knee_angle_val,
                                                    color=(0, 255, 0), arc_radius=80)  # Green

    # Draw Elbow Angle (Shoulder-Elbow-Wrist)
    if frame_landmarks.right_shoulder and frame_landmarks.right_elbow and frame_landmarks.right_wrist:
        elbow_angle_val = calculate_angle(
            frame_landmarks.right_shoulder, frame_landmarks.right_elbow, frame_landmarks.right_wrist)
        summary_image = draw_angle_on_image_precise(summary_image,
                                                    frame_landmarks.right_wrist,
                                                    frame_landmarks.right_elbow,
                                                    frame_landmarks.right_shoulder,
                                                    elbow_angle_val,
                                                    color=(255, 0, 255), arc_radius=60)  # Magenta

    # Draw Torso Angle (Shoulder-Hip-Horizontal)
    if frame_landmarks.right_hip and frame_landmarks.right_shoulder:
        horizontal_point = Point(
            frame_landmarks.right_hip.x + 100, frame_landmarks.right_hip.y)
        torso_angle_val = calculate_angle(
            frame_landmarks.right_shoulder, frame_landmarks.right_hip, horizontal_point)
        summary_image = draw_angle_on_image_precise(summary_image,
                                                    horizontal_point,
                                                    frame_landmarks.right_hip,
                                                    frame_landmarks.right_shoulder,
                                                    torso_angle_val,
                                                    color=(255, 255, 0), arc_radius=100)  # Yellow

    # Draw Ankle Angle (Knee-Ankle-Foot Index)
    if frame_landmarks.right_knee and frame_landmarks.right_ankle and frame_landmarks.right_foot_index:
        ankle_angle_val = calculate_angle(
            frame_landmarks.right_knee, frame_landmarks.right_ankle, frame_landmarks.right_foot_index)
        summary_image = draw_angle_on_image_precise(summary_image,
                                                    frame_landmarks.right_knee,
                                                    frame_landmarks.right_ankle,
                                                    frame_landmarks.right_foot_index,
                                                    ankle_angle_val,
                                                    color=(0, 255, 255), arc_radius=70)  # Cyan

    plt.figure(figsize=(16, 10))
    plt.imshow(summary_image)
    plt.title(f"Bike Fitting Summary ({model_name})")
    plt.axis('off')
    plt.show()

    image_bgr = cv2.cvtColor(summary_image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(f'cycling_summary_{model_name.lower()}.png', image_bgr)
    print(f"Summary image saved as cycling_summary_{model_name.lower()}.png")

In [ ]:
# KOPS functions

from enum import Enum


class KOPS_Point(Enum):
    heel = "heel"
    foot_index = "foot_index"
    ankle = "ankle"
    ankle_vs_index = "ankle_vs_index"


def calculate_kops_and_get_frame_user_idea(all_landmarks: AllLandmarks, video_path: str, foot_point_to_use: KOPS_Point) -> dict:
    """
    Calculates KOPS (Knee Over Pedal Spindle) for the right leg based on the user's idea:
    finds the frame where the right_foot_index has the highest X-coordinate (most right),
    and then calculates the horizontal distance between the knee and the ankle in that frame.

    Args:
        all_landmarks: An AllLandmarks object containing processed pose data.
        video_path: Path to the original video file.

    Returns:
        A dictionary containing:
        - 'kops_value': The KOPS horizontal distance for the identified frame.
        - 'identified_frame_index': Index of the frame where right_foot_index has max X.
        - 'best_kops_frame_image': The image of the identified frame with drawn landmarks and lines.
    """
    max_foot_x = -float('inf')
    identified_frame_index = -1

    # 1. Find the frame with the highest right_foot_index.x
    for i, frame_lm in enumerate(all_landmarks.frames_landmarks):
        if frame_lm.right_foot_index and frame_lm.right_foot_index.x > max_foot_x:
            max_foot_x = frame_lm.right_foot_index.x
            identified_frame_index = i

    kops_value = None
    best_kops_frame_image = None

    if identified_frame_index != -1:
        frame_lm_at_max_foot_x = all_landmarks.frames_landmarks[identified_frame_index]
        knee_point = frame_lm_at_max_foot_x.right_knee
        reference_point = None
        if foot_point_to_use == KOPS_Point.heel:
            reference_point = frame_lm_at_max_foot_x.right_heel
        elif foot_point_to_use == KOPS_Point.foot_index:
            reference_point = frame_lm_at_max_foot_x.right_foot_index
        elif foot_point_to_use == KOPS_Point.ankle:
            reference_point = frame_lm_at_max_foot_x.right_ankle
        elif foot_point_to_use == KOPS_Point.ankle_vs_index:

            a = frame_lm_at_max_foot_x.right_ankle
            b = frame_lm_at_max_foot_x.right_foot_index
            c = a + b
            reference_point = Point(c.x / 2, c.y / 2)

        if knee_point and reference_point:
            kops_value = reference_point.x - knee_point.x

            # Retrieve the image for visualization
            image = get_image_at_index(identified_frame_index)
            if image is not None:
                # Re-process with MediaPipe to get results for drawing (using static_image_mode for efficiency)
                with mp_pose.Pose(static_image_mode=True, model_complexity=2) as pose_static:
                    # MediaPipe expects BGR, convert back if needed for drawing tools
                    image_bgr_for_mp = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    image_bgr_for_mp.flags.writeable = False
                    results_static = pose_static.process(image_bgr_for_mp)

                    if results_static.pose_landmarks:
                        # , cv2.COLOR_RGB2BGR) # For drawing, ensure it's modifiable BGR
                        image_to_draw = image_bgr_for_mp
                        image_to_draw.flags.writeable = True

                        # Draw landmarks
                        mp_drawing.draw_landmarks(
                            image_to_draw, results_static.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                        # Draw KOPS lines
                        knee_x, knee_y = int(knee_point.x), int(knee_point.y)
                        ankle_x, ankle_y = int(
                            reference_point.x), int(reference_point.y)

                        # Draw vertical line from knee
                        # Green line for knee
                        cv2.line(image_to_draw, (knee_x, 0),
                                 (knee_x, VIDEO_HEIGHT), (0, 255, 0), 2)
                        # Draw vertical line from ankle (pedal spindle proxy)
                        # Blue line for pedal spindle
                        cv2.line(image_to_draw, (ankle_x, 0),
                                 (ankle_x, VIDEO_HEIGHT), (255, 0, 0), 2)

                        # Draw horizontal line connecting the two vertical lines at a mid-point
                        # using a vertical level to ensure line is straight
                        # Adjust y for visibility
                        line_y_level = min(knee_y, ankle_y) - 30
                        if line_y_level < 0:
                            # Fallback if too high
                            line_y_level = max(knee_y, ankle_y) + 30

                        # Yellow line for distance
                        cv2.line(image_to_draw, (knee_x, line_y_level),
                                 (ankle_x, line_y_level), (0, 255, 255), 2)

                        # Put text for KOPS value
                        text_pos = (min(knee_x, ankle_x), line_y_level - 10)
                        cv2.putText(image_to_draw, f"KOPS: {kops_value:.2f} px", text_pos,
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

                        # Convert back to RGB for matplotlib
                        best_kops_frame_image = cv2.cvtColor(
                            image_to_draw, cv2.COLOR_BGR2RGB)

    results = {
        'kops_value': kops_value,
        'identified_frame_index': identified_frame_index,
        'best_kops_frame_image': best_kops_frame_image
    }

    return results

# Process the Video


In [ ]:
all_frames_landmarks.clear()  # Clear any previous data

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Use a context manager for MediaPipe Pose for proper resource release
with mp_pose.Pose(min_detection_confidence=0.75, min_tracking_confidence=0.75, model_complexity=2) as pose:

    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        raise TypeError(
            f"Error opening video stream or file for MediaPipe: {VIDEO_PATH}")

    i = 0
    first_image_mp = None
    first_frame_landmarks_mp = None

    j = 0

    print("Processing video with MediaPipe...")
    while cap.isOpened():
        ret, image = cap.read()
        if not ret:
            break
        i += 1
        CURR_PERCENTAGE = i/VIDEO_LENGTH*100
        if i % 100 == 0:
            print(
                f"Processing frame {i}/{VIDEO_LENGTH}: ({CURR_PERCENTAGE:.2f}%)")
        # Cut off the last 2 seconds of the video (if too long or irrelevant motion)
        if i > VIDEO_LENGTH - 2*VIDEO_FPS and VIDEO_LENGTH > 2*VIDEO_FPS:  # only apply if video is long enough
            break
        if j > (VIDEO_FPS * VIDEO_DURATION):
            break
        j += 1  # counter for actually processed frames

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False  # To improve performance

        results = pose.process(image_rgb)

        # Extract and store landmarks
        frame_lm = extract_landmarks_mediapipe(
            results, VIDEO_WIDTH, VIDEO_HEIGHT)
        if frame_lm:
            all_frames_landmarks.append_frame(frame_lm)
            if first_image_mp is None:
                # Store the first image with landmarks drawn for visualization
                image_rgb.flags.writeable = True
                mp_drawing.draw_landmarks(
                    image_rgb, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                first_image_mp = image_rgb
                first_frame_landmarks_mp = frame_lm
        # else:
        #     print(f"No landmarks detected for frame {i}") # Optional: log when no landmarks are found

    cap.release()
    print("MediaPipe processing finished.")
    print(f"Processed {j} frames")

# Convert all stored landmarks from GPU (if applicable) to CPU
# This step is important if you were using a GPU for MediaPipe and the Point objects
# retained GPU tensor references. For typical CPU-only Mediapipe usage, it might be
# redundant but harmless.
for frame_lm in all_frames_landmarks.frames_landmarks:
    frame_lm.from_gpu_to_cpu()

LANDMARKS_FILE = "all_frames_landmarks_mediapipe.pkl"
# Save the processed landmarks to a file using joblib
joblib.dump(all_frames_landmarks, LANDMARKS_FILE)

In [ ]:
# load from file
all_frames_landmarks = joblib.load(LANDMARKS_FILE)

# Show an Image with all Points


In [ ]:
if first_image_mp is not None and first_frame_landmarks_mp is not None:
    plot_image_with_points(
        first_image_mp, first_frame_landmarks_mp, with_foot_details=True)
else:
    print("No frames with detectable landmarks were processed by MediaPipe.")

In [ ]:
print("\n--- Inspecting First Frame Landmarks (MediaPipe) for Missing Points ---")
print(f"Nose: {first_frame_landmarks_mp.nose}")
print(f"Right Shoulder: {first_frame_landmarks_mp.right_shoulder}")
print(f"Right Elbow: {first_frame_landmarks_mp.right_elbow}")
print(f"Right Wrist: {first_frame_landmarks_mp.right_wrist}")
print(f"Right Hip: {first_frame_landmarks_mp.right_hip}")
print(f"Right Knee: {first_frame_landmarks_mp.right_knee}")
print(f"Right Ankle: {first_frame_landmarks_mp.right_ankle}")
# Crucial for ankle angle
print(f"Right Foot Index: {first_frame_landmarks_mp.right_foot_index}")
print(f"Right Heel: {first_frame_landmarks_mp.right_heel}")
print("----------------------------------------------------------------------")

# Calculate the Angles


In [ ]:
# Get landmark series for calculations

dynamic_angles = defaultdict(list)

right_wrist_series = all_frames_landmarks.get_landmark_series("right_wrist")
right_elbow_series = all_frames_landmarks.get_landmark_series("right_elbow")
right_shoulder_series = all_frames_landmarks.get_landmark_series(
    "right_shoulder")
right_hip_series = all_frames_landmarks.get_landmark_series("right_hip")
right_knee_series = all_frames_landmarks.get_landmark_series("right_knee")
right_ankle_series = all_frames_landmarks.get_landmark_series("right_ankle")
right_heel_series = all_frames_landmarks.get_landmark_series("right_heel")
right_foot_index_series = all_frames_landmarks.get_landmark_series(
    "right_foot_index")

In [ ]:
# Calculate angles for each frame
for i in range(len(all_frames_landmarks.frames_landmarks)):
    frame_lm = all_frames_landmarks.frames_landmarks[i]

    # Upper Body Angles
    if frame_lm.right_hip and frame_lm.right_shoulder and frame_lm.right_elbow:
        shoulder_angle = calculate_angle(
            frame_lm.right_hip, frame_lm.right_shoulder, frame_lm.right_elbow)
        dynamic_angles["Shoulder Angle (Hip-Shoulder-Elbow)"].append(
            shoulder_angle)

    if frame_lm.right_shoulder and frame_lm.right_elbow and frame_lm.right_wrist:
        elbow_angle = calculate_angle(
            frame_lm.right_shoulder, frame_lm.right_elbow, frame_lm.right_wrist)
        dynamic_angles["Elbow Angle (Shoulder-Elbow-Wrist)"].append(elbow_angle)

    # Lower Body Angles
    if frame_lm.right_hip and frame_lm.right_knee and frame_lm.right_ankle:
        knee_angle = calculate_angle(
            frame_lm.right_hip, frame_lm.right_knee, frame_lm.right_ankle)
        dynamic_angles["Knee Angle (Hip-Knee-Ankle)"].append(knee_angle)

    # Torso Angle (e.g., Shoulder-Hip-Horizontal)
    # This is a bit more complex as it requires a horizontal reference.
    # For simplicity, let's use the angle formed by hip, shoulder and a point horizontally from shoulder
    if frame_lm.right_hip and frame_lm.right_shoulder:
        # Create an artificial point horizontally from the shoulder
        horizontal_point = Point(
            frame_lm.right_hip.x + 100, frame_lm.right_hip.y)
        torso_angle = calculate_angle(
            frame_lm.right_shoulder, frame_lm.right_hip,  horizontal_point)
        dynamic_angles["Torso Angle (Shoulder-Hip-Horizontal)"].append(torso_angle)

    # Ankle Angle (Knee-Ankle-Foot Index) # NOT USED IN BIKE FITTING
    # if frame_lm.right_knee and frame_lm.right_ankle and frame_lm.right_foot_index:
    #     ankle_angle = calculate_angle(
    #         frame_lm.right_knee, frame_lm.right_ankle, frame_lm.right_foot_index)
    #     dynamic_angles["Ankle Angle (Knee-Ankle-Foot Index)"].append(ankle_angle)

# Calculate the Knee Angle (Hip-Knee-Ankle) peaks
peak_detection_distance = 20
knee_angles = np.array(dynamic_angles["Knee Angle (Hip-Knee-Ankle)"])
# Adjust distance as needed
peaks, _ = find_peaks(knee_angles, distance=peak_detection_distance)
max_knee_angles = knee_angles[peaks]

# drop all peaks, that are below the mean of the knee angles
mean_knee_angle = np.mean(knee_angles)
new_peaks = [peak for peak in peaks if knee_angles[peak] > mean_knee_angle]
peaks = new_peaks
max_knee_angles = knee_angles[peaks]
#

In [ ]:
# Find the frames for min/max knee extension
knee_angles_list = dynamic_angles.get("Knee Angle (Hip-Knee-Ankle)", [])
foot_bottom_index_mp = -1
foot_top_index_mp = -1

if knee_angles_list:
    # Max knee angle is usually at the bottom of the pedal stroke (most extended)
    foot_bottom_index_mp = np.argmax(knee_angles_list)
    # Min knee angle is usually at the top of the pedal stroke (most flexed)
    foot_top_index_mp = np.argmin(knee_angles_list)

    print(
        f"\nFoot at Bottom (Max Knee Extension) Frame Index (MediaPipe): {foot_bottom_index_mp}")
    print(
        f"Foot at Top (Min Knee Extension) Frame Index (MediaPipe): {foot_top_index_mp}")
    show_video_at_index(foot_bottom_index_mp,
                        "Foot at Bottom (Max Knee Extension)")
    show_video_at_index(foot_top_index_mp, "Foot at Top (Min Knee Extension)")
else:
    print("\nCould not calculate knee angles with MediaPipe to determine foot bottom/top frames.")

# Results


In [ ]:
# get the video at a custom index, and apply the landmarks at that index
def get_video_at_index(index: int, landmarks: AllLandmarks):
    """Retrieves a specific frame from the video and applies landmarks."""
    frame = get_image_at_index(index)
    if frame is not None:
        # Convert to RGB for plotting
        #        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Get the landmarks for this index
        if 0 <= index < len(landmarks.frames_landmarks):
            frame_landmarks = landmarks.frames_landmarks[index]
            return frame, frame_landmarks
    return None, None


def show_video_with_landmarks(index: int, landmarks: AllLandmarks, with_foot_details=False):
    """Displays a specific frame from the video with landmarks."""
    frame_rgb, frame_landmarks = get_video_at_index(index, landmarks)
    if frame_rgb is not None and frame_landmarks is not None:
        plot_image_with_points(frame_rgb, frame_landmarks, with_foot_details)
    else:
        print(f"Could not retrieve or display frame at index {index}")


show_video_with_landmarks(min(int(VIDEO_DURATION * VIDEO_FPS), 200),  # dont go over the max index
                          all_frames_landmarks, with_foot_details=True)

## KOPS - Knee over Pedal Spindle


In [ ]:
# can be: heel, foot_index or ankle, ankle_vs_index
foot_point_to_use = KOPS_Point.foot_index

kops_analysis_results = calculate_kops_and_get_frame_user_idea(
    all_frames_landmarks, VIDEO_PATH, foot_point_to_use)

print("\n--- KOPS Analysis (User's Idea) ---")
if kops_analysis_results['kops_value'] is not None:
    print(
        f"KOPS horizontal distance (at max right foot X): {kops_analysis_results['kops_value']:.2f} pixels")
    print("Negative value: Knee is forward of estimated pedal spindle.")
    print("Positive value: Knee is behind estimated pedal spindle.")

    if kops_analysis_results['identified_frame_index'] != -1:
        print(
            f"\nDisplaying frame where right foot is furthest to the right (Frame {kops_analysis_results['identified_frame_index']})")
        if kops_analysis_results['best_kops_frame_image'] is not None:
            plt.figure(figsize=(10, 8))
            plt.imshow(kops_analysis_results['best_kops_frame_image'])
            plt.title(
                f"KOPS Visualization (Frame {kops_analysis_results['identified_frame_index']})")
            plt.axis('off')
            plt.show()
        else:
            print("Failed to retrieve or process the identified frame for visualization.")
else:
    print("Could not calculate KOPS. Ensure right_foot_index, right_knee, and right_ankle landmarks are detected.")

## Calculated Angles


In [ ]:
print("--- MediaPipe Calculated Angles ---")
if dynamic_angles:
    for angle_name, angles in dynamic_angles.items():
        print_angle_stats(angles, angle_name)
else:
    print("No MediaPipe angles data available. Please ensure the MediaPipe processing steps were run successfully.")

## Summary Image (Only first frame)


In [ ]:
if first_image_mp is not None:
    show_summarized_image(first_image_mp, first_frame_landmarks_mp, dynamic_angles,
                          foot_bottom_index_mp, foot_top_index_mp, "MediaPipe", angles_text=False)

else:
    print("Cannot show MediaPipe summary image as no initial frame was processed.")

## Angles over Time


In [ ]:
optimal_ranges = {
    # show the bottom angle
    "Knee Angle (Hip-Knee-Ankle)": OPTIMAL_KNEE_EXTENSION_BOTTOM,
    # "Knee Extension Top": OPTIMAL_KNEE_EXTENSION_TOP, # not usefull here
    "Shoulder Angle (Hip-Shoulder-Elbow)": OPTIMAL_SHOULDER_ANGLE,
    "Torso Angle (Shoulder-Hip-Horizontal)": OPTIMAL_TORSO_ANGLE_TO_HORIZONTAL,
    "Elbow Angle (Shoulder-Elbow-Wrist)": OPTIMAL_ELBOW_ANGLE,
}

plot_angles_over_time(dynamic_angles, "MediaPipe",
                      separate_figures=False, optimal_ranges=optimal_ranges, knee_peaks=peaks, max_knee_angles=max_knee_angles)

## Angles AVGs


In [ ]:
print("\n--- Cycling Angle Summary ---")

# Calculate average angles (handle cases where angle list might be empty)
avg_shoulder_angle = np.mean(dynamic_angles.get("Shoulder Angle (Hip-Shoulder-Elbow)", [
                             0])) if dynamic_angles.get("Shoulder Angle (Hip-Shoulder-Elbow)") else 0
avg_elbow_angle = np.mean(dynamic_angles.get("Elbow Angle (Shoulder-Elbow-Wrist)",
                          [0])) if dynamic_angles.get("Elbow Angle (Shoulder-Elbow-Wrist)") else 0
avg_torso_angle = np.mean(dynamic_angles.get("Torso Angle (Shoulder-Hip-Horizontal)",
                          [0])) if dynamic_angles.get("Torso Angle (Shoulder-Hip-Horizontal)") else 0
avg_knee_angle = np.mean(dynamic_angles.get("Knee Angle (Hip-Knee-Ankle)",
                         [0])) if dynamic_angles.get("Knee Angle (Hip-Knee-Ankle)") else 0
# avg_ankle_angle = np.mean(dynamic_angles.get("Ankle Angle (Knee-Ankle-Foot Index)",
#                           [0])) if dynamic_angles.get("Ankle Angle (Knee-Ankle-Foot Index)") else 0

# Get specific knee angles from identified frames
knee_angles_list = dynamic_angles.get("Knee Angle (Hip-Knee-Ankle)", [])
knee_angle_bottom = knee_angles_list[foot_bottom_index_mp] if 0 <= foot_bottom_index_mp < len(
    knee_angles_list) else float('nan')
knee_angle_top = knee_angles_list[foot_top_index_mp] if 0 <= foot_top_index_mp < len(
    knee_angles_list) else float('nan')

# Avg of the knee angles peaks
avg_knee_angle_peaks = np.mean(
    max_knee_angles) if max_knee_angles.size > 0 else float('nan')

print(f'Knee Angle (Bottom of Stroke): {knee_angle_bottom:.2f}°')
print(f'Knee Angle (Top of Stroke): {knee_angle_top:.2f}°')
print(f'Knee Angle (Avg): {avg_knee_angle:.2f}°')
print(f'Knee Angle Peaks (Avg): {avg_knee_angle_peaks:.2f}°')
print(f'Shoulder Angle (Avg): {avg_shoulder_angle:.2f}°')
print(f'Elbow Angle (Avg): {avg_elbow_angle:.2f}°')
print(f'Torso Angle (Avg): {avg_torso_angle:.2f}°')
# print(f'Ankle Angle (Avg): {avg_ankle_angle:.2f}°')
print("-----------------------------\n")

## Recommendations


In [ ]:
recommendations = get_bike_fit_recommendations(
    dynamic_angles, avg_knee_angle_peaks)
for i, rec in enumerate(recommendations):
    print(rec)